Multi-Driver Lap Delta Telemetry Analysis

This notebook engineers telemetry data from the Fast-F1 API for multiple drivers to compute lap time delta traces. It utilizes a structured workflow for data acquisition, alignment, and delta calculation, producing visualizations that identify where time is gained or lost across a lap. Additional stacked telemetry plots (speed, throttle, brake, RPM, etc.) provide context for performance differences between drivers.

NOTE: This notebook will not compile as standalone. Please pull down the project from GitHub (git clone https://github.com/yourusername/f1-driving-style-analytics-tool.git) and install the necessary dependencies. Further instructions are included in the README.md.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [1]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

logging.getLogger('fastf1').setLevel(logging.ERROR)

In this section, I import Python libraries for data visualization, numerical analysis, and working with the Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [2]:
from src.data import f1_data
from src.utils import f1_constants
from src.preprocessing import telemetry_cleaning, telemetry_processing

import pandas as pd

pd.set_option('display.max_rows', None) # reset_option to compact dataframe view
pd.set_option('display.max_columns', None)

The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

These session parameters were chosen for this analysis based on the following criteria:

- Location: Melbourne (rainy conditions)
- Green Flag Laps: Remove safety car and VSC distortions
- Single-compound Windows: Avoid mixed group comparisons
- Minimize Outliers: Tire age approx. 20 laps

In [3]:
year = 2025
grand_prix = f1_constants.F1Constants.LOCATIONS["Australia"]
session_type = f1_constants.F1Constants.SESSIONS["R"]

safety_car_laps = [1, 2, 34]

session = f1_data.F1Session(year, grand_prix, session_type)

req         WARNING 	DEFAULT CACHE ENABLED! (1.3 GB) /Users/phillipsmith/Library/Caches/fastf1
core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached 

All drivers who participated in the specific location's Grand Prix will be analyzed and assigned variables to be identified by their three-letter name code.

Constants for telemetry data is also initialized as variables for ease of use during visualization.

Sector timestamps variables are set for easy replacment during telemetry filtering.

In [4]:
"""DRIVER CONSTANTS"""
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
albon = f1_constants.F1Constants.DRIVERS["Alexander Albon"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
hamilton = f1_constants.F1Constants.DRIVERS["Lewis Hamilton"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
hadjar = f1_constants.F1Constants.DRIVERS["Isack Hadjar"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
doohan = f1_constants.F1Constants.DRIVERS["Jack Doohan"]
bortoleto = f1_constants.F1Constants.DRIVERS["Gabriel Bortoleto"]
antonelli = f1_constants.F1Constants.DRIVERS["Andrea Kimi Antonelli"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
ocon = f1_constants.F1Constants.DRIVERS["Esteban Ocon"]
bearman = f1_constants.F1Constants.DRIVERS["Oliver Bearman"]

"""THE 6 KEY TELEMETRY TRACE CONSTANTS"""
speed = f1_constants.F1Constants.TELEMETRY_COLUMNS["Speed (m/s)"]
throttle = f1_constants.F1Constants.TELEMETRY_COLUMNS["Throttle (%)"]
brakes = f1_constants.F1Constants.TELEMETRY_COLUMNS["BrakesApplied"]
rpm = f1_constants.F1Constants.TELEMETRY_COLUMNS["RPM"]
gear = f1_constants.F1Constants.TELEMETRY_COLUMNS["nGear"]
# steering = f1_constants.F1Constants.TELEMETRY_COLUMNS["Steering Wheel Angle (°)"]

"""SECTOR TIMESTAMP VARIABLES"""
s1_start = 'Sector1Start'
s1_end_s2_start = 'Sector1End_Sector2Start'
s2_end_s3_start = 'Sector2End_Sector3Start'
s3_end = 'Sector3End'

This code retrieves circuit corner data (Turn, X/Y coordinates, Angle, and Distance) to place markers on stacked telemetry and delta plots.

In [5]:
corner_position = session.get_circuit_info().corners
corner_position_cleaned = telemetry_cleaning.clean_circuit_corner_data(corner_position)

critical_turn = [None]
radius = 0

Retrieve the fastest lap telemetry for Driver 1 and Driver 2. 

This function processes the raw, accurate telemetry data to extract the relevant information for the fastest lap, including speed, throttle, brake, RPM, and other parameters. The resulting dataframes are structured to facilitate direct comparison between the two drivers.

In [6]:
"""SELECT TWO DRIVERS"""
driver_1 = hamilton
driver_2 = antonelli

"""GENERATE FASTEST LAP TELEMETRY DATAFRAMES"""
driver_1_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=driver_1,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_start,
    end=s3_end
)

driver_1_fastest_telemetry = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=driver_1_processed_data,
    driver_code=driver_1,
    corner_position=corner_position_cleaned, # not used
    critical_turn=critical_turn[0], # not used
    radius=radius, # not used
    start=s1_start,
    end=s3_end
)

driver_2_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=driver_2,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_start,
    end=s3_end
)

driver_2_fastest_telemetry = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=driver_2_processed_data,
    driver_code=driver_2,
    corner_position=corner_position_cleaned, # not used
    critical_turn=critical_turn[0], # not used
    radius=radius, # not used
    start=s1_start,
    end=s3_end
)